
<a href="https://colab.research.google.com/github/google-research/text-to-text-transfer-transformer/blob/master/notebooks/t5-trivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

Initial setup taken from t5 repository notebook.


In [0]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

BASE_DIR = "gs://fiery-lcm-000001" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models1")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  #tensorflow_gcs_config.configure_gcs_from_colab_auth('/device:CPU:0')
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
     |████████████████████████████████| 153kB 2.6MB/s 
     |████████████████████████████████| 1.1MB 10.7MB/s 
     |████████████████████████████████| 3.3MB 14.5MB/s 
     |████████████████████████████████| 296kB 46.9MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 3.0MB 42.5MB/s 
     |████████████████████████████████| 665kB 48.4MB/s 
     |████████████████████████████████| 890kB 48.6MB/s 
     |████████████████████████████████| 3.8MB 43.5MB/s 
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
Setting up GCS access...
Running on TPU: grpc://10.14.56.218:8470
Instructions for updating:
non-resource variables are not supported in th

Loading IMDB dataset.

https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews (From here the dataset is downloaded)

In [0]:
 !unzip ./imdb-dataset-of-50k-movie-reviews

Archive:  ./imdb-dataset-of-50k-movie-reviews.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import csv
import pandas as pd
import re
from numpy.random import RandomState


df =pd.read_csv('./IMDB Dataset.csv')
print(df.head())
rng = RandomState()

train = df.sample(frac=0.9, random_state=rng)
test = df.loc[~df.index.isin(train.index)]


print(train.head())
print(test.head())
def foo_bar(x):
    #x = re.sub('\s*', '\s', x)
    x = re.sub('\t','\s',x)
    return x

train = train.applymap(foo_bar)
test = test.applymap(foo_bar)

train.to_csv("imdb-train.csv",index=False, header=False)
test.to_csv("imdb-test.csv",index=False, header=False)

print(train.columns)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                                  review sentiment
46642  for those of you who were desperate to find ou...  negative
10460  I stumbled on this series rather by accident. ...  positive
9729   A great suspenseful thriller the acting is fir...  positive
42283  I'll be honest,I finally checked this movie no...  positive
20207  THE PLOT: A trucker (Kristofferson) battles a ...  negative
                                               review sentiment
0   One of the other reviewers has mentioned that ...  positive
7   This show was an amazing, fresh & innovative i...  negative
40  It had all the clichés o

In [0]:
# Writing the csv file to a tsv file.
def conv_csv_to_tsv(filename):
  with open(filename+'.csv','r') as csvin, open(filename+'.tsv', 'w') as tsvout:
    csvin = csv.reader(csvin)
    tsvout = csv.writer(tsvout, delimiter='\t')

    for row in csvin:
        tsvout.writerow(row)
conv_csv_to_tsv('imdb-train')
conv_csv_to_tsv('imdb-test')

In [0]:
# !gsutil cp ./imdb-train.tsv gs://fiery-lcm-000001/data/imdb-train.tsv
# !gsutil cp ./imdb-test.tsv gs://fiery-lcm-000001/data/imdb-test.tsv

Copying file://./imdb-train.tsv [Content-Type=text/tab-separated-values]...
\
Operation completed over 1 objects/56.9 MiB.                                     
Copying file://./imdb-test.tsv [Content-Type=text/tab-separated-values]...
-
Operation completed over 1 objects/6.3 MiB.                                      


In [0]:
import os
DATA_DIR = os.path.join(BASE_DIR, "data")
nq_tsv_path = {
    "train": os.path.join(DATA_DIR, "imdb-train.tsv"),
    "test": os.path.join(DATA_DIR, "imdb-test.tsv")
}
def imdb_dataset_fn(split, shuffle_files=False):
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  # Split each "<review>\t<sentiment>" example into (review, sentiment) tuple.
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Map each tuple to a {"review": ... "statement": ...} dict.
  ds = ds.map(lambda *ex: dict(zip(["review", "sentiment"], ex)))
  return ds

# print("A few raw validation examples...")
# with open('somefile.txt', 'a') as the_file:
#   for ex in tfds.as_numpy(nq_dataset_fn("train").take(10000)):
#     the_file.write(ex['review'].decode('utf-8'))

A few raw validation examples...


In [0]:
def imdb_preprocessor(ds):
  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    text = tf.strings.regex_replace(text,"'\s*'", r"\s")
    return text

  def to_inputs_and_targets(ex):
    """Map {"question": ..., "answer": ...}->{"inputs": ..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["", normalize_text(ex["review"])]),
        "targets": normalize_text(ex["sentiment"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
import t5.data
from t5.data import postprocessors as t5_postprocessors
from t5.evaluation import metrics as t5_metrics
TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask
TaskRegistry.remove("imdb_custom5")
t5.data.TaskRegistry.add(
    "imdb_custom5",
    # Supply a function which returns a tf.data.Dataset.
    dataset_fn=imdb_dataset_fn,
    splits=["train", "test"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[imdb_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path=t5.data.DEFAULT_SPM_PATH,
    # Lowercase targets before computing metrics.
    postprocess_fn=[t5.data.postprocessors.lower_text], 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy]
)

In [0]:
t5.data.MixtureRegistry.remove("trivia_all")
t5.data.MixtureRegistry.add(
    "trivia_all",
    ["imdb_custom5"],
     default_rate=1.0
)

## Define Model

In [0]:
MODEL_SIZE = "small" #@param["small", "base", "large", "3B", "11B"]
# Public GCS path for T5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE)

if ON_CLOUD and MODEL_SIZE == "3B":
  tf.logging.warn(
      "The `3B` model is too large to use with the 5GB GCS free tier. "
      "Make sure you have at least 25GB on GCS before continuing."
  )
elif ON_CLOUD and MODEL_SIZE == "11B":
  raise ValueError(
      "The `11B` parameter is too large to fine-tune on the `v2-8` TPU "
      "provided by Colab. Please comment out this Error if you're running "
      "on a larger TPU."
  )

# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 128, "targets": 32},
    learning_rate_schedule=0.003,
    #save_checkpoints_steps=5000,
    #keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [0]:
from google.colab import auth
auth.authenticate_user()

Before we continue, let's load a [TensorBoard](https://www.tensorflow.org/tensorboard) visualizer so that we can keep monitor our progress. The page should automatically update as fine-tuning and evaluation proceed.

In [0]:
if ON_CLOUD:
  %reload_ext tensorboard
  import tensorboard as tb
tb.notebook.start("--logdir " + MODELS_DIR)

## Train



In [0]:
import gin
with gin.unlock_config():
  gin.parse_config_file("gs://t5-data/pretrained_models/base/operative_config.gin")

In [0]:
FINETUNE_STEPS = 25000 #@param {type: "integer"}
import transformers
model.train(
    mixture_or_task_name="trivia_all",
    steps=FINETUNE_STEPS,
    split="train",
)

## Evaluate

We now evaluate on the validation sets of the tasks in our mixture. Accuracy results will be logged and added to the TensorBoard above.

In [0]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = train_batch_size * 4
model.eval(
    mixture_or_task_name="trivia_all",
    checkpoint_steps="all",
    split="test"
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://fiery-lcm-000001/models1/small', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.14.56.218:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.14.56.218:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.14.56.218:8470', '_evaluation_master': 'grpc://10.14.56.218:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replica

TypeError: ignored